<a href="https://colab.research.google.com/github/CynthiaEspinola/Hito1_G.E/blob/main/HITO_2_GESTION_E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hito 2** - Cynthia Espinola

In [1]:
pip install scipy

Opcion 1 : **Black  Black-Scholes**

In [2]:
import math
from scipy.stats import norm
from datetime import datetime as dt
import datetime
import yfinance as yf


#definimos de que "Opcion" debe traer la informacion
e = "BCH" # Especificacion del simbolo del activo, es decir, BCH (Banco de Chile)
t = yf.Ticker(e) # Creamos un objeto Ticker para el activo subyacente.
S = t.info["currentPrice"]


# para calcula la volatilidad, debemos aplicar los siguientes pasos :
# 1. Agrupar los datos más relevantes de un activo en un lapso de tiempo específico / Definimos las fechas
start_date ="2023-01-01"
end_date = dt.now()


#2. Calcular la rentabilidad dividiendo el precio del activo en x día, entre el precio original del activo.
rent = yf.download(e,start_date,end_date)

#3.Calcular volatilidad, sacando la raíz cuadrada de la rentabilidad.

rent["Returns"] = rent["Adj Close"].pct_change()
sigma=rent["Returns"].std()

# posteriormente obtenemos la informacion restante sobre la opcion para luego reemplzarla en la formula de Black-Scholes.

# Precio de ejercicio de la opción
opcion = yf.Ticker("BCH240119C00017500")
K = opcion.info["strikePrice"]

# Período hasta el vencimiento en años
D_Termino = opcion.info["expireDate"]
End_Time = datetime_obj = dt.utcfromtimestamp(D_Termino)
T= ((End_Time - end_date)).days/365


# Calcula r como la tasa de interés de un activo libre de riesgo.

# Buscamos las siglas bono del Tesoro de EE. UU.
treasury_bond = "^TNX"

# Definimos el periodo que deseo obtener la tasa de interés libre de riesgo.
fecha_inicio = "2023-01-01"

# Crea un objeto Ticker para el bono del Tesoro.
treasury_bond_ticker = yf.Ticker(treasury_bond)

# Obtenemos el historial de precios del bono del Tesoro en el periodo determinado
bond_history = treasury_bond_ticker.history(period="1d", start=fecha_inicio, end=dt.now())


# Finalmente obtenemos la tasa de interes
risk_free_rate = bond_history.iloc[0]["Close"] / 100  # Dividimos por 100 para obtener el valor en tanto por uno.

r = risk_free_rate

############################################################################################################################################################################

# Calcula el precio de una opción utilizando la fórmula Black-Scholes.
def black_scholes(S, K, T, r, sigma, option_type='call'):


    d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    option_price = S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)
    return option_price

#  Calculamos la función de distribución acumulativa estándar de una variable aleatoria normal.

def norm_cdf(x):
    from scipy.stats import norm
    return norm.cdf(x)


call_option_price = black_scholes(S, K, T, r, sigma, option_type='call')


print("Precio de una opción de compra del Banco Chile :",call_option_price)

[*********************100%%**********************]  1 of 1 completed
Precio de una opción de compra del Banco Chile : 2.1820943883819908


**Opción 2 - Valorización Binomial**

In [1]:
import math
from scipy.stats import norm
from datetime import datetime as dt
import datetime
import yfinance as yf
#Utilizaremos los mismos datos obtenidos previamente.

#definimos de que "Opcion" debe traer la informacion
e = "BCH" # Especificacion del simbolo del activo, es decir, BCH (Banco de Chile)
t = yf.Ticker(e) # Creamos un objeto Ticker para el activo subyacente.
S = t.info["currentPrice"]


# para calcula la volatilidad, debemos aplicar los siguientes pasos :
# 1. Agrupar los datos más relevantes de un activo en un lapso de tiempo específico / Definimos las fechas
start_date ="2023-01-01"
end_date = dt.now()


#2. Calcular la rentabilidad dividiendo el precio del activo en equis día, entre el precio original del activo.
rent = yf.download(e,start_date,end_date)

#3.Calcular volatilidad, sacando la raíz cuadrada de la rentabilidad.

rent["Returns"] = rent["Adj Close"].pct_change()
sigma=rent["Returns"].std()

# posteriormente obtenemos la informacion restante sobre la opcion para luego reemplzarla en la formula de Black-Scholes.

# Precio de ejercicio de la opción
opcion = yf.Ticker("BCH240119C00017500")
K = opcion.info["strikePrice"]

# Período hasta el vencimiento en años
D_Termino = opcion.info["expireDate"]
End_Time = datetime_obj = dt.utcfromtimestamp(D_Termino)
T= ((End_Time - end_date)).days/365


# Calcula r como la tasa de interés de un activo libre de riesgo.

# Buscamos las siglas bono del Tesoro de EE. UU.
treasury_bond = "^TNX"

# Definimos el periodo que deseo obtener la tasa de interés libre de riesgo.
fecha_inicio = "2023-01-01"

# Crea un objeto Ticker para el bono del Tesoro.
treasury_bond_ticker = yf.Ticker(treasury_bond)

# Obtenemos el historial de precios del bono del Tesoro en el periodo determinado
bond_history = treasury_bond_ticker.history(period="1d", start=fecha_inicio, end=dt.now())


# Finalmente obtenemos la tasa de interes
risk_free_rate = bond_history.iloc[0]["Close"] / 100  # Dividimos por 100 para obtener el valor en tanto por uno.

r = risk_free_rate


def binomial_option_pricing(S, K, T, r, sigma, n):
    f_t = T / n
    u = 1 + (r * f_t) + (sigma * (f_t ** 0.5))
    d = 1 + (r * f_t) - (sigma * (f_t ** 0.5))
    q = (1 + r - d) / (u - d)

    # Definimos el inicio del árbol de precios de la opción ("BCH240119C00017500"")
    option_prices = [0] * (n + 1)

    # Calculamos los precios de la opción en la última capa del árbol
    for i in range(n + 1):
      option_prices[i] = max(0, S * (u ** (n - i)) * (d ** i) - K)

    # Hacemos un barrido hacia atrás en el árbol para calcular el precio actual de la opción
    for step in range(n - 1, -1, -1):
        for i in range(step + 1):
            option_prices[i] = (q * option_prices[i] + (1 - q) * option_prices[i + 1]) / (1 + r)

    return option_prices[0]

n=10 # n: Número de pasos en el modelo binomial
option_price = binomial_option_pricing(S, K, T, r, sigma, n)

print("Precio de la opción:" , option_price)




[*********************100%%**********************]  1 of 1 completed
Precio de la opción: 7.470202381430388
